In [ ]:
# Cell 1: Importing required libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import cv2

In [ ]:
# Cell 2: Loading dataset paths
train_dir = './seg_train'
test_dir = './seg_test'

# Define image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Load the datasets
train_dataset = image_dataset_from_directory(train_dir, shuffle=True, image_size=IMG_SIZE, batch_size=BATCH_SIZE)
test_dataset = image_dataset_from_directory(test_dir, shuffle=True, image_size=IMG_SIZE, batch_size=BATCH_SIZE)

# Cell 3: Model definition using ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_dataset.class_names), activation='softmax')(x)  # Number of classes = 6 (buildings, forest, etc.)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of ResNet50 to avoid retraining them
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Cell 4: Compiling the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Cell 5: Training the model
EPOCHS = 25
history = model.fit(train_dataset, validation_data=test_dataset, epochs=EPOCHS)

In [ ]:
# Save the trained model to a .h5 file
model.save('scene_classification_model.h5')
print("Model saved as 'scene_classification_model.h5'")


In [ ]:
# Cell 6: Grad-CAM Implementation
def get_img_array(img_path, size):
    img = load_img(img_path, target_size=size)
    array = img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
# Cell 7: Updated Visualizing Grad-CAM with Prediction
def display_gradcam(image_path, model, last_conv_layer_name='conv5_block3_out'):
    img_array = get_img_array(image_path, size=IMG_SIZE)
    
    # Get the model prediction
    preds = model.predict(img_array)
    pred_class_index = np.argmax(preds[0])  # Index of predicted class
    pred_class = train_dataset.class_names[pred_class_index]  # Predicted class name

    # Display the predicted class
    print(f"Predicted Class: {pred_class} (confidence: {preds[0][pred_class_index] * 100:.2f}%)")

    # Generate Grad-CAM heatmap
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

    # Load and preprocess the original image
    img = load_img(image_path)
    img = img_to_array(img)

    # Resize heatmap to match the image size
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)

    # Apply heatmap to the original image
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.4 + img
    superimposed_img = np.uint8(superimposed_img)

    # Display the images
    plt.figure(figsize=(10, 10))
    
    plt.subplot(1, 2, 1)
    plt.imshow(img.astype('uint8'))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(superimposed_img)
    plt.title('Grad-CAM')
    plt.axis('off')

    plt.show()
